In [1]:
import json
import finnhub
import yfinance as yf
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from datetime import datetime

# Set up Finnhub API and get financials data

In [2]:
finnhub_client = finnhub.Client(api_key='c4o2l1aad3ie88nplncg')

financials_payload = finnhub_client.financials_reported(symbol='AAPL', freq='annual')
print(json.dumps(financials_payload, indent=4))

{
    "cik": "320193",
    "data": [
        {
            "accessNumber": "0000320193-20-000096",
            "symbol": "AAPL",
            "cik": "320193",
            "year": 2020,
            "quarter": 0,
            "form": "10-K",
            "startDate": "2019-09-29 00:00:00",
            "endDate": "2020-09-26 00:00:00",
            "filedDate": "2020-10-30 00:00:00",
            "acceptedDate": "2020-10-29 18:06:25",
            "report": {
                "bs": [
                    {
                        "unit": "usd",
                        "label": "Cash and cash equivalents",
                        "value": 38016000000,
                        "concept": "CashAndCashEquivalentsAtCarryingValue"
                    },
                    {
                        "unit": "usd",
                        "label": "Marketable securities",
                        "value": 52927000000,
                        "concept": "MarketableSecuritiesCurrent"
                    },
 

# A simple example for Bokeh line charts

In [3]:
# this code is needed to ensure proper Bokeh graph rendering via the notebook
output_notebook()

Loading BokehJS ...

In [4]:
# sample data
x = [1, 2, 3, 4, 5]
y = [2, 1, 5, 1, 7]

In [5]:
# initialize the figure object
p = figure(title='Simple Line Chart', x_axis_label='x', y_axis_label='y')

In [6]:
# add a line renderer to the figure object
p.line(x, y, legend_label='Temp', line_width=2)

GlyphRenderer(id='1039', ...)

In [7]:
# show results
show(p)

In [8]:
# add more lines to the same chart
y2 = [2, 3, 4, 5, 6]
y3 = [4, 5, 5, 7, 2]

p.line(x, y2, legend_label='Rate', line_color='red', line_width=2)
p.line(x, y3, legend_label='Object', line_color='green', line_width=2)

GlyphRenderer(id='1160', ...)

In [9]:
# show the updated chart with multiple lines
show(p)

# Unpack data from the payload for financial charts

In [10]:
def unpack_list_dict(payload, key, condition=True, reverse=True):
    """This helper function unpacks a list of dictionaries with a given key and condition."""
    
    l = [item[key] for item in payload if condition]
    if reverse:
        l.reverse()
    return l

In [11]:
# get the list of years from financials_payload, reversed
list_years = unpack_list_dict(financials_payload['data'], 'year')
print(list_years)

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [12]:
# get the list of reports from financials_payload, reversed
list_reports = unpack_list_dict(financials_payload['data'], 'report')
print(json.dumps(list_reports, indent=4))

[
    {
        "bs": [
            {
                "unit": "usd",
                "label": "Available-for-sale Securities, Debt Securities, Noncurrent",
                "value": 25391000000,
                "concept": "AvailableForSaleSecuritiesDebtSecuritiesNoncurrent"
            },
            {
                "unit": "usd",
                "label": "Tangible assets that are held by an entity for use in the production or supply of goods and services, for rental to others, or for administrative purposes and that are expected to provide economic benefit for more than one year; net of accumulated depreciation. Examples include land, buildings, and production equipment. Also includes software programs or applications for internal use (that is, not to be sold, leased or otherwise marketed to others) that qualify for capitalization.",
                "value": 4768000000,
                "concept": "aapl:PropertyPlantAndEquipmentAndCapitalizedSoftwareNet"
            },
            {
 

In [13]:
def report_transform(list_reports, sections=['bs', 'ic', 'cf'], key_name='concept', key_value='value', missing=None):
    """
    This helper function transforms the list of historical financial reports to a dictionary to 
    hold the data for different metrics.
    
    The key of each item is the name of the metric, and the value is a list of historical values
    of the same metric taken from the list of reports.
    """
    
    # initialize the metrics payload by section, with each section has its own dictionary of metric-value pairs
    metrics = {}
    for section in sections:
        metrics[section] = {}
    
    # loop through the code below twice, 
    # with the first time initializing the list of historical values for each
    # metric that has ever appeared in any historical financial report;
    # and the second time appending historical values to the initialized list for each metric
    tasks = ['init', 'append']
    for task in tasks:
    
        # appeared in any historical financial report
        for report_id in range(len(list_reports)):

            # within each report, loop through different sections of the report, 
            # namely "balance sheet", "income", and "cash flow"
            for section in sections:    

                # initialize the list of historical values for each new metric found in this
                # report-section
                if task == 'init':
                    # loop through different metrics available in the current report-section
                    for field_id in range(len(list_reports[report_id][section])):

                        # gets the name for the current metric
                        metric_name = list_reports[report_id][section][field_id][key_name]

                        # if the name of this metric does not exist yet in the dictionary (from earlier reports),
                        # initialize a new dictionary item with the initial value set to be an empty list
                        if metric_name not in metrics[section].keys():
                            metrics[section][metric_name] = []
                
                # append values to the lists of historical values already initialized
                else:
                    # loop through all initialized metrics
                    for metric_name in metrics[section].keys():
                        
                        # append the corresponding value if the metric exists in the current report-section;
                        # otherwise append a value representing missing values 
                        _value = missing
                        for data in list_reports[report_id][section]:
                            if data[key_name] == metric_name:
                                _value = data[key_value]
                        metrics[section][metric_name].append(_value)
    
            
    # return the cleaned up dictionary
    return metrics


# test the report transform helper function
aapl_metrics = report_transform(list_reports)
print(json.dumps(aapl_metrics, indent=4))

{
    "bs": {
        "AvailableForSaleSecuritiesDebtSecuritiesNoncurrent": [
            25391000000,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null
        ],
        "aapl:PropertyPlantAndEquipmentAndCapitalizedSoftwareNet": [
            4768000000,
            7777000000,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null
        ],
        "Goodwill": [
            741000000,
            896000000,
            1135000000,
            1577000000,
            4616000000,
            5116000000,
            5414000000,
            5717000000,
            null,
            null,
            null
        ],
        "IntangibleAssetsNetExcludingGoodwill": [
            342000000,
            3536000000,
            4224000000,
            417900000

In [14]:
def get_metric_hist_values(payload, key_year='year', key_report='report', 
                           sections=['bs', 'ic', 'cf'], key_name='concept', 
                           key_value='value'):
    """
    This function takes the payload of a sequence of historical financial reports as input,
    and spits out a dictionary, with items being:
    'year': [year_value_1, year_value_2, ..., year_value_N],
    'metric_1_name': [metric_1_value_year_1, metric_1_value_year_2, ..., metric_1_value_year_N],
    'metric_2_name': [metric_2_value_year_1, metric_2_value_year_2, ..., metric_2_value_year_N],
    ...
    """
    
    # get list of years and reports from the input payload
    list_years = unpack_list_dict(payload['data'], key_year)
    list_reports = unpack_list_dict(payload['data'], key_report)
    
    # get a dictionary of metrics from the input payload
    metrics = report_transform(list_reports, sections=sections, key_name=key_name, key_value=key_value)
    
    # add year values to the same dictionary
    metrics['year'] = list_years

    return metrics


# test function 'get_metric_hist_values'
financials_payload_aapl = finnhub_client.financials_reported(symbol='AAPL', freq='annual')
metrics_aapl = get_metric_hist_values(financials_payload_aapl)
print(json.dumps(metrics_aapl, indent=4))

{
    "bs": {
        "AvailableForSaleSecuritiesDebtSecuritiesNoncurrent": [
            25391000000,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null
        ],
        "aapl:PropertyPlantAndEquipmentAndCapitalizedSoftwareNet": [
            4768000000,
            7777000000,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null
        ],
        "Goodwill": [
            741000000,
            896000000,
            1135000000,
            1577000000,
            4616000000,
            5116000000,
            5414000000,
            5717000000,
            null,
            null,
            null
        ],
        "IntangibleAssetsNetExcludingGoodwill": [
            342000000,
            3536000000,
            4224000000,
            417900000

# Test the consistency of core financial metrics across different stocks

In [15]:
# Google
financials_payload_googl = finnhub_client.financials_reported(symbol='GOOGL', freq='annual')
metrics_googl = get_metric_hist_values(financials_payload_googl)
print(json.dumps(metrics_googl, indent=4))

{
    "bs": {
        "CashAndCashEquivalentsAtCarryingValue": [
            16549000000,
            12918000000,
            10715000000,
            16701000000,
            18498000000,
            26465000000
        ],
        "AvailableForSaleSecuritiesCurrent": [
            56517000000,
            73415000000,
            91156000000,
            null,
            null,
            null
        ],
        "CashCashEquivalentsAndShortTermInvestments": [
            73066000000,
            86333000000,
            101871000000,
            109140000000,
            119675000000,
            136694000000
        ],
        "AccountsReceivableNetCurrent": [
            11556000000,
            14137000000,
            18336000000,
            20838000000,
            25326000000,
            30930000000
        ],
        "goog:ReceivableUnderReverseRepurchaseAgreements": [
            450000000,
            0,
            null,
            null,
            null,
            nu

In [16]:
# CVS
financials_payload_cvs = finnhub_client.financials_reported(symbol='CVS', freq='annual')
metrics_cvs = get_metric_hist_values(financials_payload_cvs)
print(json.dumps(metrics_cvs, indent=4))

{
    "bs": {
        "CashAndCashEquivalentsAtCarryingValue": [
            1427000000,
            1413000000,
            1375000000,
            4089000000,
            2481000000,
            2459000000,
            3371000000,
            1696000000,
            4059000000,
            5683000000,
            7854000000
        ],
        "ShortTermInvestments": [
            4000000,
            5000000,
            5000000,
            88000000,
            34000000,
            88000000,
            87000000,
            111000000,
            2522000000,
            2373000000,
            3000000000
        ],
        "AccountsReceivableNetCurrent": [
            4925000000,
            6047000000,
            6473000000,
            8729000000,
            9687000000,
            11888000000,
            12164000000,
            13181000000,
            17631000000,
            19617000000,
            21742000000
        ],
        "InventoryNet": [
            10695000000

In [17]:
# EPD
financials_payload_epd = finnhub_client.financials_reported(symbol='EPD', freq='annual')
metrics_epd = get_metric_hist_values(financials_payload_epd)
print(json.dumps(metrics_epd, indent=4))

{
    "bs": {
        "CashAndCashEquivalentsAtCarryingValue": [
            65500000,
            19800000,
            16100000,
            56900000,
            74400000,
            19000000,
            63100000,
            5100000,
            344800000,
            334700000,
            1059900000
        ],
        "RestrictedCashAndInvestmentsCurrent": [
            98700000,
            38500000,
            4300000,
            65600000,
            0,
            15900000,
            null,
            null,
            null,
            null,
            null
        ],
        "AccountsNotesAndLoansReceivableNetCurrent": [
            3800100000,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null
        ],
        "DueFromRelatedPartiesCurrent": [
            36800000,
            43500000,
            2500000,
            6800000,
          

In [18]:
# test the consistency of several key metrics across the payloads of different stocks
print("Google:", metrics_googl['ic'].get('WeightedAverageNumberOfDilutedSharesOutstanding', 0))
print("CVS:", metrics_cvs['ic'].get('WeightedAverageNumberOfDilutedSharesOutstanding', 0))
print("EPD", metrics_epd['ic'].get('WeightedAverageNumberOfDilutedSharesOutstanding', 0))

Google: 0
CVS: [1377000000, 1347000000, 1280000000, 1226000000, 1169000000, 1126000000, 1079000000, 1024000000, 1044000000, 1305000000, 1314000000]
EPD 0


In [19]:
def search_metric(metrics_dict, metric_name):
    for key in metrics_dict.keys():
        if metric_name in key.lower():
            print(key)

In [20]:
search_metric(metrics_epd['cf'], 'oper')

GainLossOnSaleOfPropertyPlantEquipment
NetCashProvidedByUsedInOperatingActivities
PaymentsToAcquirePropertyPlantAndEquipment
GainLossOnDispositionOfProperty
IncreaseDecreaseInOperatingCapital
OtherOperatingActivitiesCashFlowStatement
NetCashProvidedByUsedInOperatingActivitiesContinuingOperations
NetCashProvidedByUsedInInvestingActivitiesContinuingOperations
NetCashProvidedByUsedInFinancingActivitiesContinuingOperations
OperatingLeaseRightOfUseAssetAmortizationExpense


In [21]:
metrics_googl['ic']['EarningsPerShareDiluted']

[None, None, None, 43.7, 49.16, None]

# Test the yfinance API for getting historical price data

In [22]:
aapl = yf.Ticker('AAPL')
print(aapl)

yfinance.Ticker object <AAPL>


In [23]:
# get stock info
aapl.info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 147000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

In [24]:
aapl_quote_history = aapl.history(interval='1mo', start='2010-01-01', end='2021-09-23')
print(aapl_quote_history)

                  Open        High         Low       Close        Volume  \
Date                                                                       
2010-01-01    6.544802    6.611039    5.833991    5.889494  1.516899e+10   
2010-02-01    5.898999    6.291510    5.852389    6.274644  1.077608e+10   
2010-03-01    6.309298    7.282295    6.300098    7.206245  1.215417e+10   
2010-04-01    7.280145    8.354948    7.137247    8.006289  1.236713e+10   
2010-05-01    8.090619    8.214506    6.109975    7.877192  1.808265e+10   
...                ...         ...         ...         ...           ...   
2021-05-07         NaN         NaN         NaN         NaN           NaN   
2021-06-01  124.892880  137.204435  122.945792  136.755112  1.606590e+09   
2021-07-01  136.395644  149.775590  135.556889  145.641785  1.918976e+09   
2021-08-01  146.141042  153.260380  144.283824  151.602859  1.460744e+09   
2021-09-01  152.830002  157.259995  141.270004  142.830002  1.708233e+09   

           

In [25]:
aapl_quote_history_payload = dict(aapl_quote_history['Close'])
dict(aapl_quote_history['Close'])

{Timestamp('2010-01-01 00:00:00'): 5.8894944190979,
 Timestamp('2010-02-01 00:00:00'): 6.274643898010254,
 Timestamp('2010-03-01 00:00:00'): 7.206244945526123,
 Timestamp('2010-04-01 00:00:00'): 8.0062894821167,
 Timestamp('2010-05-01 00:00:00'): 7.87719202041626,
 Timestamp('2010-06-01 00:00:00'): 7.713135242462158,
 Timestamp('2010-07-01 00:00:00'): 7.888535499572754,
 Timestamp('2010-08-01 00:00:00'): 7.454631805419922,
 Timestamp('2010-09-01 00:00:00'): 8.701156616210938,
 Timestamp('2010-10-01 00:00:00'): 9.229514122009277,
 Timestamp('2010-11-01 00:00:00'): 9.541375160217285,
 Timestamp('2010-12-01 00:00:00'): 9.891263961791992,
 Timestamp('2011-01-01 00:00:00'): 10.405205726623535,
 Timestamp('2011-02-01 00:00:00'): 10.831140518188477,
 Timestamp('2011-03-01 00:00:00'): 10.68701457977295,
 Timestamp('2011-04-01 00:00:00'): 10.736689567565918,
 Timestamp('2011-05-01 00:00:00'): 10.66616153717041,
 Timestamp('2011-06-01 00:00:00'): 10.293275833129883,
 Timestamp('2011-07-01 00:00:

In [26]:
foo = list(aapl_quote_history_payload.keys())[0]

In [27]:
foo.strftime('%Y-%m-%d')

'2010-01-01'

In [28]:
from datetime import datetime

In [29]:
datetime.strptime('2010-01-01', '%Y-%m-%d')

datetime.datetime(2010, 1, 1, 0, 0)

# prep quote & fundamental data for valuation graphs

In [30]:
financials_payload_aapl = finnhub_client.financials_reported(symbol='AAPL', freq='annual')
metrics_aapl = get_metric_hist_values(financials_payload_aapl)
print(json.dumps(metrics_aapl, indent=4))

{
    "bs": {
        "AvailableForSaleSecuritiesDebtSecuritiesNoncurrent": [
            25391000000,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null
        ],
        "aapl:PropertyPlantAndEquipmentAndCapitalizedSoftwareNet": [
            4768000000,
            7777000000,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null,
            null
        ],
        "Goodwill": [
            741000000,
            896000000,
            1135000000,
            1577000000,
            4616000000,
            5116000000,
            5414000000,
            5717000000,
            null,
            null,
            null
        ],
        "IntangibleAssetsNetExcludingGoodwill": [
            342000000,
            3536000000,
            4224000000,
            417900000

In [31]:
valuation_payload = dict(zip(metrics_aapl['year'], metrics_aapl['ic']['EarningsPerShareDiluted']))
valuation_payload

{2010: 15.15,
 2011: 27.68,
 2012: 44.15,
 2013: 39.75,
 2014: 6.45,
 2015: 9.22,
 2016: 8.31,
 2017: 9.21,
 2018: 11.91,
 2019: 11.89,
 2020: 3.28}

In [32]:
timestamps = aapl_quote_history_payload.keys()

quote_data = {}
metric_data = {}

for t in timestamps:
    if not np.isnan(aapl_quote_history_payload[t]):
        new_t = datetime.strptime(t.strftime('%Y-%m-%d'), '%Y-%m-%d')
        if new_t.year < datetime.utcnow().year:
            quote_data[new_t] = aapl_quote_history_payload[t]
            metric_data[new_t] = valuation_payload.get(new_t.year, None)

In [33]:
quote_data

{datetime.datetime(2010, 1, 1, 0, 0): 5.8894944190979,
 datetime.datetime(2010, 2, 1, 0, 0): 6.274643898010254,
 datetime.datetime(2010, 3, 1, 0, 0): 7.206244945526123,
 datetime.datetime(2010, 4, 1, 0, 0): 8.0062894821167,
 datetime.datetime(2010, 5, 1, 0, 0): 7.87719202041626,
 datetime.datetime(2010, 6, 1, 0, 0): 7.713135242462158,
 datetime.datetime(2010, 7, 1, 0, 0): 7.888535499572754,
 datetime.datetime(2010, 8, 1, 0, 0): 7.454631805419922,
 datetime.datetime(2010, 9, 1, 0, 0): 8.701156616210938,
 datetime.datetime(2010, 10, 1, 0, 0): 9.229514122009277,
 datetime.datetime(2010, 11, 1, 0, 0): 9.541375160217285,
 datetime.datetime(2010, 12, 1, 0, 0): 9.891263961791992,
 datetime.datetime(2011, 1, 1, 0, 0): 10.405205726623535,
 datetime.datetime(2011, 2, 1, 0, 0): 10.831140518188477,
 datetime.datetime(2011, 3, 1, 0, 0): 10.68701457977295,
 datetime.datetime(2011, 4, 1, 0, 0): 10.736689567565918,
 datetime.datetime(2011, 5, 1, 0, 0): 10.66616153717041,
 datetime.datetime(2011, 6, 1,

In [34]:
metric_data

{datetime.datetime(2010, 1, 1, 0, 0): 15.15,
 datetime.datetime(2010, 2, 1, 0, 0): 15.15,
 datetime.datetime(2010, 3, 1, 0, 0): 15.15,
 datetime.datetime(2010, 4, 1, 0, 0): 15.15,
 datetime.datetime(2010, 5, 1, 0, 0): 15.15,
 datetime.datetime(2010, 6, 1, 0, 0): 15.15,
 datetime.datetime(2010, 7, 1, 0, 0): 15.15,
 datetime.datetime(2010, 8, 1, 0, 0): 15.15,
 datetime.datetime(2010, 9, 1, 0, 0): 15.15,
 datetime.datetime(2010, 10, 1, 0, 0): 15.15,
 datetime.datetime(2010, 11, 1, 0, 0): 15.15,
 datetime.datetime(2010, 12, 1, 0, 0): 15.15,
 datetime.datetime(2011, 1, 1, 0, 0): 27.68,
 datetime.datetime(2011, 2, 1, 0, 0): 27.68,
 datetime.datetime(2011, 3, 1, 0, 0): 27.68,
 datetime.datetime(2011, 4, 1, 0, 0): 27.68,
 datetime.datetime(2011, 5, 1, 0, 0): 27.68,
 datetime.datetime(2011, 6, 1, 0, 0): 27.68,
 datetime.datetime(2011, 7, 1, 0, 0): 27.68,
 datetime.datetime(2011, 8, 1, 0, 0): 27.68,
 datetime.datetime(2011, 9, 1, 0, 0): 27.68,
 datetime.datetime(2011, 10, 1, 0, 0): 27.68,
 datet

In [35]:
def average_price_multiple(quote_data, metric_data):
    """
    This helper function computes the average price multiple of the metric included in the input metric data dictionary,
    as well as the "fair price" for each timestamp by multiplying the metric value by the average price multiple.
    
    Input:
        quote_data: dictionary. The key is a datetime object, and the value is the market closing price 
                    on the day specified by the same datetime object
        metric_data: dictionary. The key is a datetime object, and the value is the metric value for the 
                     year specified by the same datetime object
                     
    
    """
    
    average = np.average(np.array(list(quote_data.values())) / np.array(list(metric_data.values())))
    
    fair_price = {}
    for key in metric_data.keys():
        fair_price[key] = metric_data[key] * average

    return average, fair_price 


average_pe_ratio, fair_price_pe_ratio = average_price_multiple(quote_data, metric_data)
print("average_pe_ratio =", average_pe_ratio)
print("fair_price_pe_ratio =")
fair_price_pe_ratio

average_pe_ratio = 4.7581876686032665
fair_price_pe_ratio =


{datetime.datetime(2010, 1, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 2, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 3, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 4, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 5, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 6, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 7, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 8, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 9, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 10, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 11, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2010, 12, 1, 0, 0): 72.08654317933949,
 datetime.datetime(2011, 1, 1, 0, 0): 131.7066346669384,
 datetime.datetime(2011, 2, 1, 0, 0): 131.7066346669384,
 datetime.datetime(2011, 3, 1, 0, 0): 131.7066346669384,
 datetime.datetime(2011, 4, 1, 0, 0): 131.7066346669384,
 datetime.datetime(2011, 5, 1, 0, 0): 131.7066346669384,
 datetime.datetime(2011, 6, 

In [36]:
aapl.splits

Date
2014-06-09    7.0
2020-08-31    4.0
Name: Stock Splits, dtype: float64

# TODO - figure out how to restore the nominal the historical prices, if there have been stock splits;

Otherwise things like P/E ratios won't be correct for points in time before those splits

In [39]:
aapl.cashflow

,2020-09-26,2019-09-28,2018-09-29,2017-09-30
Investments,5.335000e+09,5.809300e+10,3.084500e+10,-3.354200e+10
Change To Liabilities,-1.981000e+09,-2.548000e+09,9.172000e+09,8.373000e+09
Total Cashflows From Investing Activities,-4.289000e+09,4.589600e+10,1.606600e+10,-4.644600e+10
Net Borrowings,2.499000e+09,-7.819000e+09,4.320000e+08,2.901400e+10
Total Cash From Financing Activities,-8.682000e+10,-9.097600e+10,-8.787600e+10,-1.797400e+10
Change To Operating Activities,8.810000e+08,-8.960000e+08,3.001600e+10,-8.480000e+09
Issuance Of Stock,8.800000e+08,7.810000e+08,6.690000e+08,5.550000e+08
Net Income,5.741100e+10,5.525600e+10,5.953100e+10,4.835100e+10
Change In Cash,-1.043500e+10,2.431100e+10,5.624000e+09,-1.950000e+08
Repurchase Of Stock,-7.599200e+10,-6.971400e+10,-7.526500e+10,-3.477400e+10
